# Contextual AI-Powered Insights Generator

## Overview
This notebook demonstrates an advanced AI-powered system for generating contextual insights 
about payment technologies, leveraging simplified retrieval-augmented generation (RAG) with Ollama.

## Key Features:
- Semantic search across payment technology contexts
- Context-aware response generation
- Flexible querying mechanism
- Fallback response handling

## Technologies Used:
- LangChain
- Ollama Embeddings
- FAISS Vector Store
- Retrieval-Augmented Generation (RAG)

## Use Cases:
- Payment gateway research
- Financial technology insights
- Contextual Q&A about payment systems

In [3]:
#!ollama pull mxbai-embed-large

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaEmbeddings, ChatOllama

# Payment-related context data
payment_contexts = [
    "Credit card transactions require secure tokenization and encryption.",
    "Payment gateways support multiple payment methods like Visa, Mastercard, and PayPal.",
    "Stripe offers comprehensive payment processing for online businesses.",
    "Modern Treasury simplifies complex payment operations across different banks.",
    "Spreedly allows secure storage and transaction of payment methods across multiple gateways.",
]

# Configure embeddings
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Initialize FAISS vector store with payment-related text data
vectorstore = FAISS.from_texts(
    payment_contexts,
    embedding=ollama_embeddings,
)

# Convert vector store to retriever
retriever = vectorstore.as_retriever()

# Define prompt template with a more specific payment-focused context
template = """You are an expert in payment systems and financial technology. 
Answer the question based strictly on the following context:

Context:
{context}

Question: {question}

If the context does not provide sufficient information, clearly state that you cannot provide a comprehensive answer based on the available information."""

# Initialize chat prompt from template
prompt = ChatPromptTemplate.from_template(template)

# Initialize Ollama chat model
ollama_model = ChatOllama(
    base_url="http://localhost:11434",
    model='llama3.2:1b',
    temperature=0.5,
    num_predict=512
)

# Format retrieved documents
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

# Build retrieval chain with context-aware responses
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_model
    | StrOutputParser()
)






In [ ]:
if __name__ == "__main__":  
    # Queries with context retrieval
    print("Query 1: Payment Gateway Methods")
    result1 = retrieval_chain.invoke("What payment methods do modern payment gateways support?")
    print(result1)

    print("\nQuery 2: Payment Security")
    result2 = retrieval_chain.invoke("How do payment systems ensure transaction security?")
    print(result2)




Query 1: Payment Gateway Methods
Based on the provided context, it appears that modern payment gateways typically support multiple payment methods, including:

1. Visa
2. Mastercard
3. PayPal (or other digital wallets)

These are the most commonly supported payment methods by major payment gateways such as Stripe and Spreedly.

However, I cannot provide a comprehensive answer without additional context or information about specific payment gateways. If you have more details about the payment gateway in question, I would be happy to try and provide a more detailed response.

Query 2: Payment Security
I can provide an informative response based on general knowledge of payment systems and financial technology.

Payment systems employ various mechanisms to ensure transaction security, including:

1. **Tokenization**: Replacing sensitive card information with unique tokens, which are used for transactions. This makes it difficult for attackers to access the actual card data.
2. **Encryption

In [6]:
if __name__ == "__main__":    
    # Query without context (fallback)
    print("\nQuery 3: Without Context")
    fallback_chain = prompt | ollama_model | StrOutputParser()
    result3 = fallback_chain.invoke({"question": "What payment methods do modern payment gateways support?", "context": ""})
    print(result3)


Query 3: Without Context
I can provide an overview of commonly supported payment methods by modern payment gateways.

Modern payment gateways typically support various payment methods, including:

1. Credit/Debit Cards: Visa, Mastercard, American Express, Discover
2. Debit Cards: Similar to credit cards but require immediate access to the account balance
3. Online Banking and Digital Wallets:
	* PayPal (via PayPal Payments Standard)
	* Apple Pay, Google Pay, Samsung Pay, and other contactless payment methods
4. Mobile Payments:
	* Apple Pay, Google Pay, Samsung Pay, and other mobile wallets
5. Alternative Payment Methods:
	* Bitcoin (for cryptocurrency transactions)
	* Cryptocurrencies like Ethereum, Litecoin, and others
6. Digital Currencies:
	* Ethereum, Litecoin, and other blockchain-based currencies
7. Prepaid Cards: Prepaid cards can be used for various payments, including online purchases

Please note that the specific payment methods supported by a particular payment gateway ma